# GraphRAG Relevance, Factuality and Synthesis Evaluation

## Step 0: environment set up

In [8]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index
import pandas as pd
from conn import (
    connect2Googlesheet,
    retrieval_rel_docs,
    get_concatenate_df,
    apply_metric,
)
from libs import context_builder

# Force reload of the .env file
load_dotenv()

True

In [11]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD"),
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

Connected to Neo4j database successfully.


## Step 1: Create vector index

In [12]:
create_vector_index(graph, "entities")

Index 'entities' exists but has incorrect dimensions: None. Recreating...


## Step 2: Load questions from google sheet

In [15]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
worksheet = spreadsheet.get_worksheet(2)

# Get all records as a list of dictionaries
data = worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(data)
df_MedQ.head()

FileNotFoundError: [Errno 2] No such file or directory: 'reliable-mode-399404-dc4b73f57296.json # DO NOT CHANGE HERE!!!'

## Step 3: Relevance check for top K questions

In [5]:
def retrieval_rel_docs(graph, questions, top_k=5):
    top_k_questions = questions.head(top_k)
    # Initialize a list to store the results
    results = []
    # Iterate over the top k questions
    for index, row in top_k_questions.iterrows():
        question_number = index + 1  # Assuming the question number is the index + 1
        question = row[
            "Question"
        ]  # Replace 'Question' with the actual column name for questions in df_MedQ

        # Generate response for the question
        context = context_builder(graph, question, method="vector")

        # Extract relevant documents from the response content
        # docs = response.choices[0].message.content  # Adjust this based on the actual response structure

        # Append the result to the list
        results.append(
            {
                "Question number": question_number,
                "Question": question,
                "Retrieved": context,
            }
        )

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(
        results, columns=["Question number", "Question", "Retrieved"]
    )

    return results_df


# retrieval_rel_docs (graph, questions, top_k=1)

In [6]:
# Set pandas display options to show the full text content
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
topk = 5
results_df = retrieval_rel_docs(graph, df_MedQ, top_k=topk)
results_df
# results_df.to_csv('./outputs/retrieved_docs_results.csv', index=False)

,Question number,Question,Retrieved
0,1,Does early administration of neuromuscular blocking agents increases the ventilator free days?,"Given the following context in the format [(File Name, Text),...] \n[('LSPA.pdf', ' clinical trials have shown that early mobilization in both medical and surgical critically ill patients is safe and associated with increased ventilator-free days and improved physical function at hospital discharge.22–25 Early mobilization is limited by use of deep sedation and development of delirium, which can be minimized through the use of scale- based targeted light sedation is implemented early on.26 After reviewing this literature in 2013, the Society of Critical Care Medicine (SCCM)’s')]"
1,2,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,"Given the following context in the format [(File Name, Text),...] \n[('ACURASYS.pdf', ' Steroidal compounds (vecuronium, pancuronium, and ro\xad cu\xadronium) may carry the highest risk of myopa- thy,23 although myopathy has also been reported with benzylisoquinolines, including cisatracuri- um besylate.24,25 Muscle weakness was not in- creased significantly by the use of the neuro- muscular blocking agent in our study. The short duration of use of the neuromuscular blocking agent probably explains this result. In conclusion, this multicenter trial provides evidence that the administration of a neuromuscu- lar blocking agent early in the course of severe ARDS managed with low-tidal-volume ventilation may improve outcomes. Future studies are need- ed to replicate and expand these findings before they can be widely adopted in clinical practice. Supported by the Assistance Publique–Hôpitaux de Marseille and a grant from the Ministère de')]"
2,3,"In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?","Given the following context in the format [(File Name, Text),...] \n[('ACURASYS.pdf', ' Steroidal compounds (vecuronium, pancuronium, and ro\xad cu\xadronium) may carry the highest risk of myopa- thy,23 although myopathy has also been reported with benzylisoquinolines, including cisatracuri- um besylate.24,25 Muscle weakness was not in- creased significantly by the use of the neuro- muscular blocking agent in our study. The short duration of use of the neuromuscular blocking agent probably explains this result. In conclusion, this multicenter trial provides evidence that the administration of a neuromuscu- lar blocking agent early in the course of severe ARDS managed with low-tidal-volume ventilation may improve outcomes. Future studies are need- ed to replicate and expand these findings before they can be widely adopted in clinical practice. Supported by the Assistance Publique–Hôpitaux de Marseille and a grant from the Ministère de')]"
3,4,Do patients with moderate-to-severe ARDS have a significance difference in mortality rate beween patients who recieved an early and continous cisatracurium infusion than those with usual care approach with lighter sedation targets?,"Given the following context in the format [(File Name, Text),...] \n[('ROSE.pdf', ' approach to mechanical ventilation that included lighter sedation targets. This trial had high adherence to the protocol, including minimal crossover use of neuromuscular block- ade and high adherence to the recommended ventilation and fluid strategy. The results of prespecified subgroup analyses were consistent with those of the primary analysis across sever- ity and duration of ARDS and across trial sites with different exclusion rates for previous neuro- muscular blockade use. Several factors may explain why our findings differed from those of ACURASYS, the previous multicenter trial that showed a benefit with early continuous neuromuscular blockade. First, we used a higher PEEP strategy in both groups to test our intervention in the context of best care and to reduce the likelihood of differential PEEP

In [7]:
# old results
old_results = pd.read_csv("./outputs/retrieved_docs_results.csv")
old_results

,Question number,Generated Docs
0,1,"""ACURASYS"""
1,2,"""ACURASYS"""
2,3,"""ACURASYS"""
3,4,"""ROSE"", ""ACURASYS"""
4,5,"""ARDS"", ""ACURASYS"""


In [ ]:
# load the relevant documents
# topk = 2
# results_df = pd.read_csv('./outputs/retrieved_docs_results.csv')
# relevant_docs_df = pd.read_csv('./outputs/relevant_docs.csv').rename(columns={'Docs': 'Relevant Docs'})
# concatenated_df = get_concatenate_df(results_df, relevant_docs_df, topk)
# concatenated_df.head()
# relevant_docs_df.head()
# context_builder(graph, query, method="vector")
# relevant_docs_df.head()

In [10]:
# Display the concatenated DataFrame with metrics
# concatenated_df = apply_metric(concatenated_df)
# concatenated_df
# Save the concatenated DataFrame to a CSV file
# concatenated_df.to_csv('./outputs/concatenated_results.csv', index=False)